In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
import pandas as pd
import scipy as scp
import netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import pickle
import matplotlib.gridspec as gridspec


In [ ]:
file_path1 = 'SSTs.nc'
file_path2 = 'TCs.nc'
  
#Opening dataset for tropics                                   
ds1 = xr.open_dataset(file_path1)
ds2 = xr.open_dataset(file_path2)

In [ ]:
ds2

In [ ]:
ds_grouped = ds1.groupby('time.year').mean(dim='time')                                   #Obtained Yearly Meaned Data
tropical_data = ds_grouped.sel(latitude=slice(23.5, -23.5), longitude=slice(0, 360))    #Extracted Data For the Tropics

In [ ]:

SSTs = {}

# Iterate over each year in the tropical_data
for year, data in tropical_data.groupby('year'):
    # Get the SST data for the current year
    sst_data = data['sst']
    
    # Calculate the tropical mean SST for the year
    tropical_mean_sst = sst_data.mean(dim=['latitude', 'longitude'], skipna=True)
    
    # Create a mask where SST exceeds the tropical mean SST for the current year
    up = sst_data > tropical_mean_sst
    
    # Select the SST data where the mask is True
    masked_sst = sst_data.where(up)

    # Store the masked SST data for the current year
    SSTs[int(year)] = masked_sst




In [ ]:
with open("storms_by_year.pk", "rb") as fin:  
 storms = pickle.load(fin)

In [ ]:
# Initialize a dictionary to store storm information with latitudes and longitudes
storm_info = {}

# Iterate over the years in the storms dictionary
for year, storm_ids_year in storms.items():
    # Initialize a dictionary to store storm information for the current year
    storm_info_year = {}
    # Iterate over the storm IDs for the current year
    for storm_id in storm_ids_year:
        # Find the storm data for the current storm ID
        storm_data = ds2.sel(storm=storm_id)
        # Extract latitude and longitude values
        latitudes = storm_data['lat'].values
        longitudes = storm_data['lon'].values
        # Store latitude and longitude values in the storm_info_year dictionary
        storm_info_year[storm_id] = {'lat': latitudes, 'lon': longitudes}
    # Store storm information for the current year in the storm_info dictionary
    storm_info[year] = storm_info_year


In [ ]:

# Create a figure with subplots for each year (up to 50 years)
fig, axs = plt.subplots(50, 1, figsize=(300, 600), subplot_kw={'projection': ccrs.PlateCarree()}, 
                        gridspec_kw={'width_ratios': [1], 'height_ratios': [1]*50})

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.1)

# Flatten the axs array to iterate over all subplots
axs = axs.flatten()

# Iterate over each year from 1971 to 2021 or until the 50th subplot
for i, year in enumerate(range(1971, 2022)):
    if i >= 50:
        break
    
    # Extract storm latitude and longitude data for the current year
    year_storms = storm_info.get(year, {})
    
    # Get the corresponding axis for the current subplot
    ax = axs[i]
    
    # Plot storm locations on the map for the current year
    for storm_id, data in year_storms.items():
        latitudes = data['lat']
        longitudes = data['lon']
        ax.plot(longitudes, latitudes, linestyle='-', label=f'Storm {storm_id}', linewidth=2.5)
    
    # Get the SST data for the current year
    sst_data = SSTs.get(year, None)
    
    # Add SST data if available for the current year
    if sst_data is not None:
        pcm = ax.pcolormesh(sst_data['longitude'], sst_data['latitude'], sst_data-273.15, cmap='coolwarm', alpha=0.5)
        
        # Add color bar
        cbar = fig.colorbar(pcm, ax=ax, orientation='vertical', pad=0.01)
        cbar.set_label('Sea Surface Temperature (°C)', fontsize=20)
        cbar.ax.tick_params(labelsize=15)
    
    # Add map features
    ax.coastlines()
    ax.gridlines()
    ax.set_title(f'Year {year}', fontsize=25)

# Show the plot
plt.show()

